# Datasets

Topics:
* How are numerical data returned (and expected) in **DynamicalSystems.jl**.
* Basic `Dataset` handling.
* Neighborhoods.

---

Much of the functionality of **DynamicalSystems.jl** uses numerical data. We required a `struct` that would unify behavior across all functions that either return or require numerical data.

To this end, we use a struct called simply `Dataset`.

In [1]:
using DynamicalSystems

In [2]:
x = rand(1000)
y = rand(1000)
dataset = Dataset(x,y)

2-dimensional Dataset{Float64} with 1000 points:
 0.990605  0.225951 
 0.812981  0.154526 
 0.490817  0.962962 
 0.225787  0.600326 
 0.688322  0.443765 
 0.536709  0.918189 
 0.448972  0.680773 
 0.909004  0.997239 
 0.460936  0.541494 
 0.716157  0.898359 
 0.637662  0.380442 
 0.869326  0.676887 
 0.335159  0.299579 
 ⋮                  
 0.309705  0.515526 
 0.876828  0.795993 
 0.427572  0.209889 
 0.482404  0.771648 
 0.113016  0.274178 
 0.348546  0.0380722
 0.375838  0.562152 
 0.925618  0.346873 
 0.336631  0.94357  
 0.51948   0.327394 
 0.669318  0.581128 
 0.43938   0.986355 


A `Dataset` is a subtype of `AbstractDataset`:

In [3]:
typeof(dataset) <: AbstractDataset

true

All subtypes of `AbstractDataset` contain their data in the field `data`:

In [4]:
dataset.data

1000-element Array{StaticArrays.SArray{Tuple{2},Float64,1,2},1}:
 [0.990605, 0.225951] 
 [0.812981, 0.154526] 
 [0.490817, 0.962962] 
 [0.225787, 0.600326] 
 [0.688322, 0.443765] 
 [0.536709, 0.918189] 
 [0.448972, 0.680773] 
 [0.909004, 0.997239] 
 [0.460936, 0.541494] 
 [0.716157, 0.898359] 
 [0.637662, 0.380442] 
 [0.869326, 0.676887] 
 [0.335159, 0.299579] 
 ⋮                    
 [0.309705, 0.515526] 
 [0.876828, 0.795993] 
 [0.427572, 0.209889] 
 [0.482404, 0.771648] 
 [0.113016, 0.274178] 
 [0.348546, 0.0380722]
 [0.375838, 0.562152] 
 [0.925618, 0.346873] 
 [0.336631, 0.94357]  
 [0.51948, 0.327394]  
 [0.669318, 0.581128] 
 [0.43938, 0.986355]  

We chose a vector of `SVectors` as the internal representation of the data because it gives big performance gains in many functions of the library.

# Creating a Dataset

Most functions of **DynamicalSystems.jl** that return numerical data, return it in the form of a `Dataset`.

If you already have the data though, there are many ways to create a dataset:

In [5]:
# From matrix:
m = rand(1000, 5)
data = Dataset(m)

# using re-interpret if each *row* is one variable:
m = transpose(m)
data2 = reinterpret(Dataset, m)

data2 == data

true

In [6]:
# From columns
x, y = rand(1000), rand(1000)
data = Dataset(x, y)

# All points of a dataset must be equally sized:
z = rand(100)
Dataset(x, z)

LoadError: [91mBoundsError: attempt to access 100-element Array{Float64,1} at index [101][39m

# Handling a Dataset

Special attention has been given so that subtypes of `AbstractDataset` behave exactly like some `Matrix` where each *row* is a *point* of the dataset, while each *column* is *one dynamic variable*. The reason we choose this approach is because traditionally this is how scientific data are recorded, exported and passed around.

In [7]:
# Create some random dataset:
dataset = Dataset(rand(10000, 3))

dataset[:, 2] # this is the second variable timeseries
dataset[1] == dataset[1, :] # this is the first datapoint (D-dimensional)
dataset[5, 3] # value of the third variable, at the 5th timepoint

0.8923728336240995

In [8]:
# iteration:
f = 0.0
for point in dataset
    f += mean(point)
end

# get columns
x, y = columns(dataset)
mean(x) ≈ mean(y)

# minima, maxima, etc...
mini = minima(dataset)
maxi = maxima(dataset)
mini, maxi = minmaxima(dataset)

([0.000285759, 0.00017747, 2.06445e-5], [0.999945, 1.0, 0.999852])

In [9]:
# Function minmaxima is faster than using the two individual ones
using BenchmarkTools
@btime maxima($dataset)
@btime minima($dataset)
@btime minmaxima($dataset)

  25.864 μs (1 allocation: 112 bytes)
  26.685 μs (1 allocation: 112 bytes)
  43.518 μs (2 allocations: 224 bytes)


([0.000285759, 0.00017747, 2.06445e-5], [0.999945, 1.0, 0.999852])

# I/O
Input/output functionality for an `AbstractDataset` is already achieved using base Julia, specifically `writedlm` and `readdlm`.

The thing to note is that all data of an `AbstractDataset` is contained within its field `data`.

To write and read a dataset, simply do:

In [10]:
# I will write and read using delimiter ','
writedlm("data.txt", data.data, ',')

# Don't forget to convert the matrix to a Dataset when reading
data = Dataset(readdlm("data.txt", ',', Float64))

2-dimensional Dataset{Float64} with 1000 points:
 0.623655  0.794018 
 0.327743  0.5294   
 0.744811  0.578711 
 0.450533  0.984161 
 0.113491  0.729811 
 0.964226  0.219892 
 0.982188  0.922891 
 0.205943  0.3872   
 0.657739  0.245811 
 0.462655  0.907499 
 0.293036  0.73643  
 0.383197  0.144261 
 0.248561  0.920899 
 ⋮                  
 0.43992   0.498417 
 0.55771   0.955456 
 0.92023   0.0101294
 0.856417  0.899826 
 0.523095  0.77775  
 0.123051  0.512756 
 0.201588  0.412181 
 0.956699  0.143071 
 0.98242   0.249353 
 0.50086   0.781588 
 0.472521  0.578034 
 0.648476  0.821133 


In [11]:
# delete the dummy file we created:
rm("data.txt")
isfile("data.txt")

false

# Neighborhoods 

A "neighborhood" is a collection of points that is near a given point. `Dataset`s the amazing module [`NearestNeighbors`](https://github.com/KristofferC/NearestNeighbors.jl) in order to find this neighborhood.

We use the function `neighborhood`. The call singature is:
```julia
neighborhood(point, tree, ntype)
```


`point` is simply the query point. `tree` is the structure required by [`NearestNeighbors`](https://github.com/KristofferC/NearestNeighbors.jl), and is obtained simply by:

In [12]:
tree = KDTree(dataset)

NearestNeighbors.KDTree{StaticArrays.SArray{Tuple{3},Float64,1,3},Distances.Euclidean,Float64}
  Number of points: 10000
  Dimensions: 3
  Metric: Distances.Euclidean(0.0)
  Reordered: true

The third argument to `neighborhood` is the *type* of the neighborhood. 

* There are two types of neighborhoods!

The first one is defined as the `k` nearest points to a given point. It is represented in code by:

In [13]:
mybuddies = FixedMassNeighborhood(3) # for experts: does a knn search

DynamicalSystemsBase.FixedMassNeighborhood(3)

In [14]:
point = ones(3)
n = neighborhood(point, tree, mybuddies)

3-element Array{Int64,1}:
 2230
 1635
 9531

Notice that the `neighborhood` function does not return the points themselves, but rather the indices of the points in the original data:

In [15]:
for i in n
    println(dataset[i])
end

[0.994833, 0.952813, 0.977146]
[0.980235, 0.962263, 0.987385]
[0.990768, 0.973388, 0.992335]


The second type of neighborhood is all the points that are within some given distance `ε` from a point.

In code, we represent this as:

In [16]:
where_u_at = FixedSizeNeighborhood(0.001) # for experts: does an inrange search

DynamicalSystemsBase.FixedSizeNeighborhood(0.001)

In [17]:
n2 = neighborhood(point, tree, where_u_at)

0-element Array{Int64,1}

In [18]:
plz_come_closer = FixedSizeNeighborhood(0.1)
n2 = neighborhood(point, tree, plz_come_closer)

8-element Array{Int64,1}:
 1571
 7354
  972
 2230
 1635
 8851
 9531
 1954

In [19]:
for i in n2
    println(dataset[i])
end

[0.995186, 0.936665, 0.935324]
[0.967531, 0.943782, 0.963174]
[0.961248, 0.942233, 0.946658]
[0.994833, 0.952813, 0.977146]
[0.980235, 0.962263, 0.987385]
[0.919214, 0.993461, 0.98792]
[0.990768, 0.973388, 0.992335]
[0.944716, 0.998348, 0.924357]


Okay, so points that have distance < ε are accepted as a neighborhood.

What is the "distance" though? When defining a `tree`, you can optionally give a distance function. By default this is the Euclidean distance, but others also work:

In [20]:
using Distances # to get distance functions
funky_tree = KDTree(dataset, Chebyshev())

NearestNeighbors.KDTree{StaticArrays.SArray{Tuple{3},Float64,1,3},Distances.Chebyshev,Float64}
  Number of points: 10000
  Dimensions: 3
  Metric: Distances.Chebyshev()
  Reordered: true

In [21]:
n3 = neighborhood(point, funky_tree, plz_come_closer)

12-element Array{Int64,1}:
 5147
 1571
 7354
  972
 2230
 1635
 8851
 9531
 1954
 6309
 7374
 4367

# Excluding temporal neighbors

Before moving on, let's see one last thing:

In [22]:
# the point I want the neighborhood is now part of my dataset:
point = dataset[end]

3-element StaticArrays.SArray{Tuple{3},Float64,1,3}:
 0.590523
 0.942317
 0.855156

In [23]:
# Reminder of where the tree comes from:
tree = KDTree(dataset)

# Find suuuuuuuper close neighbors:
ε = 0.000001
where_u_at = FixedSizeNeighborhood(ε)
n2 = neighborhood(point, tree, where_u_at)

# Find the nearest neighbor:
my_best_friend = FixedMassNeighborhood(1)
n3 = neighborhood(point, tree, my_best_friend)

println(n2)
println(n3)

[10000]
[10000]


In [24]:
length(dataset) == n2[1] == n3[1]

true

*Apparently my best friend is myself. Who would have thought...*

**What is happening here is that the `neighborhood` also counted the point itself, since it is also part of the dataset.**

* Almost always this behavior needs to be avoided. For this reason, there is a second method for `neighborhood`:

```julia
neighborhood(point, tree, ntype, idx::Int, w::Int = 1)
```

In this case, `idx` is the index of the point in the original data. `w` stands for the Theiler window (positive integer).

Only points that have index
`abs(i - idx) ≥ w` are returned as a neighborhood, to exclude close temporal neighbors.

* The default `w=1` is the case of exluding the `point` itself.

Let's revisit the last example (using the default value of `w = 1`):

In [25]:
point = dataset[end]
idx = length(dataset)

n2 = neighborhood(point, tree, where_u_at, idx)
n3 = neighborhood(point, tree, my_best_friend, idx)

println(n2)
println(n3)

Int64[]
[8737]


As you can see, there isn't *any* neighbor of `point` with distance `< 0.000001` in this dataset, but there is always a nearest neighbor:

In [26]:
dataset[n3[1]]

3-element StaticArrays.SArray{Tuple{3},Float64,1,3}:
 0.598972
 0.929458
 0.854145

# Documentation Strings

In [27]:
?Dataset

search: Dataset write_dataset read_dataset AbstractDataset DynamicalSystemsBase



```
Dataset{D, T} <: AbstractDataset{D,T}
```

A dedicated interface for datasets, i.e. vectors of vectors. It contains *equally-sized datapoints* of length `D`, represented by `SVector{D, T}`.

It can be used exactly like a matrix that has each of the columns be the timeseries of each of the dynamic variables. [`trajectory`](@ref) always returns a `Dataset`. For example,

```julia
ds = Systems.towel()
data = trajectory(ds, 1000) #this returns a dataset
data[:, 2] # this is the second variable timeseries
data[1] == data[1, :] # this is the first datapoint (D-dimensional)
data[5, 3] # value of the third variable, at the 5th timepoint
```

Use `Matrix(dataset)` or `reinterpret(Matrix, dataset)` and `Dataset(matrix)` or `reinterpret(Dataset, matrix)` to convert. The `reinterpret` methods are cheaper but assume that each variable/timeseries is a *row* and not column of the `matrix`.

If you have various timeseries vectors `x, y, z, ...` pass them like `Dataset(x, y, z, ...)`. You can use `columns(dataset)` to obtain the reverse, i.e. all columns of the dataset in a tuple.


In [28]:
?neighborhood

search: neighborhood AbstractNeighborhood FixedSizeNeighborhood



```
neighborhood(point, tree, ntype)
neighborhood(point, tree, ntype, n::Int, w::Int = 1)
```

Return a vector of indices which are the neighborhood of `point` in some `data`, where the `tree` was created using `tree = KDTree(data [, metric])`. The `ntype` is the type of neighborhood and can be any subtype of [`AbstractNeighborhood`](@ref).

Use the second method when the `point` belongs in the data, i.e. `point = data[n]`. Then `w` stands for the Theiler window (positive integer). Only points that have index `abs(i - n) ≥ w` are returned as a neighborhood, to exclude close temporal neighbors. The default `w=1` is the case of exluding the `point` itself.

## References

`neighborhood` simply interfaces the functions `knn` and `inrange` from [NearestNeighbors.jl](https://github.com/KristofferC/NearestNeighbors.jl) by using the argument `ntype`.


In [29]:
?AbstractNeighborhood

search: AbstractNeighborhood



```
AbstractNeighborhood
```

Supertype of methods for deciding the neighborhood of points for a given point.

Concrete subtypes:

  * `FixedMassNeighborhood(K::Int)` : The neighborhood of a point consists of the `K` nearest neighbors of the point.
  * `FixedSizeNeighborhood(ε::Real)` : The neighborhood of a point consists of all neighbors that have distance < `ε` from the point.

See [`neighborhood`](@ref) for more.
